## Assignment Of Week 5 :-

### Data Preprocessing and feature engineering
#### Resources :
https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data

## About Dataset.

#### Ames Housing Dataset

The Ames Housing dataset tells the story of hundreds of houses sold in a small city called **Ames, Iowa**.

Imagine you are walking through a neighborhood where every house has a little card describing it in detail:

- **How big is the lot?**
- **When was the house built?**
- **How many bedrooms and bathrooms does it have?**
- **Does it have a garage, a fireplace, or a swimming pool?**
- **What is the quality of the kitchen, the basement, or the roof?**

All these details—and many more—are carefully recorded.  
Each house is also labeled with the **price it finally sold for**.

Some houses are brand new with modern finishes and big open porches. Others are older, maybe needing repairs, with small yards or unfinished basements. Some are on quiet streets; others sit near busy roads or train tracks.

The dataset captures all these factors—**size, condition, age, location, and features**—so you can learn how they relate to the value of a house.

By studying this data, you can build a model that tries to answer a simple but important question:

💡 *Given all these details, how much is this house likely to sell for?*

This makes the Ames Housing dataset a great example for learning about data analysis, feature engineering, and building predictive models in real estate.


In [4]:
import pandas as pd

# Load the datasets
train_path = "train.csv"
test_path = "test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Show basic info to understand the dataset before preprocessing
train_info = train_df.info()
train_head = train_df.head()
test_info = test_df.info()
test_head = test_df.head()

(train_info, train_head, test_info, test_head)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

(None,
    Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
 0   1          60       RL         65.0     8450   Pave   NaN      Reg   
 1   2          20       RL         80.0     9600   Pave   NaN      Reg   
 2   3          60       RL         68.0    11250   Pave   NaN      IR1   
 3   4          70       RL         60.0     9550   Pave   NaN      IR1   
 4   5          60       RL         84.0    14260   Pave   NaN      IR1   
 
   LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
 0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
 1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
 2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
 3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
 4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   
 
   YrSold  SaleType  SaleCondition  SalePrice  

In [5]:
# Count missing values per column in training data
missing_values_train = train_df.isnull().sum().sort_values(ascending=False)

# Display only columns with missing values
missing_values_train[missing_values_train > 0]


PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

#### let’s do this preprocessing in code for the train dataset first.

In [6]:
# Fill categorical columns with many missing values with 'None'
for col in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']:
    train_df[col] = train_df[col].fillna('None')

# Fill LotFrontage with median
train_df['LotFrontage'] = train_df['LotFrontage'].fillna(train_df['LotFrontage'].median())

# Fill MasVnrArea with 0 (means no veneer)
train_df['MasVnrArea'] = train_df['MasVnrArea'].fillna(0)

# Fill MasVnrType with 'None'
train_df['MasVnrType'] = train_df['MasVnrType'].fillna('None')

# Fill GarageYrBlt with YearBuilt
train_df['GarageYrBlt'] = train_df['GarageYrBlt'].fillna(train_df['YearBuilt'])

# Fill Garage-related categorical columns with 'None'
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    train_df[col] = train_df[col].fillna('None')

# Fill Basement-related categorical columns with 'None'
for col in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']:
    train_df[col] = train_df[col].fillna('None')

# Fill Electrical with mode
train_df['Electrical'] = train_df['Electrical'].fillna(train_df['Electrical'].mode()[0])

# Verify no missing values remain
missing_values_after = train_df.isnull().sum().sort_values(ascending=False)
missing_values_after[missing_values_after > 0]


Series([], dtype: int64)

Feature Engineering
Let’s create new derived features that can improve model performance:

Age of the house
HouseAge = YrSold - YearBuilt

Remodeled Flag
Remodeled = (YearBuilt != YearRemodAdd)

Garage Age
GarageAge = YrSold - GarageYrBlt

Total Bathrooms
TotalBath = FullBath + (HalfBath * 0.5) + BsmtFullBath + (BsmtHalfBath * 0.5)

Total Porch Area
TotalPorchSF = OpenPorchSF + EnclosedPorch + 3SsnPorch + ScreenPorch

HasPool / HasFireplace / HasGarage flags

In [7]:
# Feature: House Age
train_df['HouseAge'] = train_df['YrSold'] - train_df['YearBuilt']

# Feature: Remodeled flag (1 if remodeled, 0 if not)
train_df['Remodeled'] = (train_df['YearBuilt'] != train_df['YearRemodAdd']).astype(int)

# Feature: Garage Age
train_df['GarageAge'] = train_df['YrSold'] - train_df['GarageYrBlt']

# Feature: Total Bathrooms
train_df['TotalBath'] = (
    train_df['FullBath'] +
    0.5 * train_df['HalfBath'] +
    train_df['BsmtFullBath'] +
    0.5 * train_df['BsmtHalfBath']
)

# Feature: Total Porch SF
train_df['TotalPorchSF'] = (
    train_df['OpenPorchSF'] +
    train_df['EnclosedPorch'] +
    train_df['3SsnPorch'] +
    train_df['ScreenPorch']
)

# Binary flags
train_df['HasPool'] = (train_df['PoolArea'] > 0).astype(int)
train_df['HasFireplace'] = (train_df['Fireplaces'] > 0).astype(int)
train_df['HasGarage'] = (train_df['GarageType'] != 'None').astype(int)

# Show sample of new features
train_df[['HouseAge', 'Remodeled', 'GarageAge', 'TotalBath', 'TotalPorchSF', 'HasPool', 'HasFireplace', 'HasGarage']].head()


,HouseAge,Remodeled,GarageAge,TotalBath,TotalPorchSF,HasPool,HasFireplace,HasGarage
0,5,0,5.0,3.5,61,0,0,1
1,31,0,31.0,2.5,0,0,1,1
2,7,1,7.0,3.5,42,0,1,1
3,91,1,8.0,2.0,307,0,1,1
4,8,0,8.0,3.5,84,0,1,1


## Ordinal mapping dictionary

In [8]:
# Ordinal mapping
quality_mapping = {
    'None': 0,
    'Po': 1,
    'Fa': 2,
    'TA': 3,
    'Gd': 4,
    'Ex': 5
}

# List of ordinal columns to encode
ordinal_cols = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
    'HeatingQC', 'KitchenQual', 'FireplaceQu',
    'GarageQual', 'GarageCond', 'PoolQC'
]

# Apply the mapping
for col in ordinal_cols:
    train_df[col] = train_df[col].map(quality_mapping)

# Show a sample of encoded columns
train_df[ordinal_cols].head()


,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond,PoolQC
0,4,3,4,3,5,4,0,3,3,0
1,3,3,4,3,5,3,3,3,3,0
2,4,3,4,3,5,4,3,3,3,0
3,3,3,3,4,4,4,4,3,3,0
4,4,3,4,3,5,4,3,3,3,0


In [9]:
# Nominal columns to one-hot encode
nominal_cols = [
    'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
    'LotConfig', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle',
    'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
    'Foundation', 'Heating', 'CentralAir', 'Functional',
    'GarageType', 'GarageFinish', 'PavedDrive', 'SaleType', 'SaleCondition'
]

# One-hot encode and drop first category to avoid multicollinearity
train_df_encoded = pd.get_dummies(train_df, columns=nominal_cols, drop_first=True)

# Show shape and sample columns after encoding
(train_df_encoded.shape, train_df_encoded.columns[:10])


((1460, 209),
 Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'Alley', 'LandSlope',
        'Condition2', 'OverallQual', 'OverallCond', 'YearBuilt'],
       dtype='object'))

In [10]:
from sklearn.preprocessing import StandardScaler

# Columns to exclude from scaling
exclude_cols = ['Id', 'SalePrice']

# Identify numeric columns
numeric_cols = train_df_encoded.select_dtypes(include=['int64', 'float64']).columns
numeric_cols_to_scale = [col for col in numeric_cols if col not in exclude_cols]

# Initialize scaler
scaler = StandardScaler()

# Fit and transform
train_df_encoded[numeric_cols_to_scale] = scaler.fit_transform(train_df_encoded[numeric_cols_to_scale])

# Show a sample of scaled features
train_df_encoded[numeric_cols_to_scale].head()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,YrSold,HouseAge,GarageAge,TotalBath,TotalPorchSF
0,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,1.052302,-0.238112,...,-0.270208,-0.068692,-0.066236,-0.087688,-1.599111,0.138777,-1.043259,-0.998630,1.642256,-0.248063
1,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,-0.689604,-0.238112,...,-0.270208,-0.068692,-0.066236,-0.087688,-0.489110,-0.614439,-0.183465,-0.011700,0.368581,-0.828163
2,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,1.052302,-0.238112,...,-0.270208,-0.068692,-0.066236,-0.087688,0.990891,0.138777,-0.977121,-0.922712,1.642256,-0.428750
3,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.689604,-0.238112,...,-0.270208,-0.068692,-0.066236,-0.087688,-1.599111,-1.367655,1.800676,-0.884753,-0.268257,2.091355
4,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,1.052302,-0.238112,...,-0.270208,-0.068692,-0.066236,-0.087688,2.100892,0.138777,-0.944052,-0.884753,1.642256,-0.029337


## Summary of Preprocessing Done
1. Missing values handled

2. Ordinal features label-encoded

3. Nominal features one-hot encoded

4. Engineered features created

5. Numerical features scaled

This dataset is now clean and ready for modeling (e.g., regression, tree-based models).

## Test Data Preprocessing Plan
Handle Missing Values

Same strategy as training set.

Feature Engineering

Create the same derived columns.

Ordinal Encoding

Use same mapping dictionary.

One-Hot Encoding

Align columns with training set (add missing dummies if needed).

Scaling Numerical Features

Use the same scaler fitted on training data.

## Handling Missing Values

In [11]:
# Fill categorical columns with many missing values with 'None'
for col in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']:
    test_df[col] = test_df[col].fillna('None')

# Fill LotFrontage with median of train set (to avoid data leakage)
test_df['LotFrontage'] = test_df['LotFrontage'].fillna(train_df['LotFrontage'].median())

# Fill MasVnrArea with 0
test_df['MasVnrArea'] = test_df['MasVnrArea'].fillna(0)

# Fill MasVnrType with 'None'
test_df['MasVnrType'] = test_df['MasVnrType'].fillna('None')

# Fill GarageYrBlt with YearBuilt
test_df['GarageYrBlt'] = test_df['GarageYrBlt'].fillna(test_df['YearBuilt'])

# Fill Garage-related categorical columns with 'None'
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    test_df[col] = test_df[col].fillna('None')

# Fill Basement-related categorical columns with 'None'
for col in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']:
    test_df[col] = test_df[col].fillna('None')

# Fill Electrical with mode
test_df['Electrical'] = test_df['Electrical'].fillna(train_df['Electrical'].mode()[0])

# Verify no missing values remain
missing_test_after = test_df.isnull().sum()
missing_test_after[missing_test_after > 0]


MSZoning        4
Utilities       2
Exterior1st     1
Exterior2nd     1
BsmtFinSF1      1
BsmtFinSF2      1
BsmtUnfSF       1
TotalBsmtSF     1
BsmtFullBath    2
BsmtHalfBath    2
KitchenQual     1
Functional      2
GarageCars      1
GarageArea      1
SaleType        1
dtype: int64

In [12]:
# Categorical columns: fill with mode
categorical_fill_mode = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'KitchenQual', 'Functional', 'SaleType']
for col in categorical_fill_mode:
    test_df[col] = test_df[col].fillna(train_df[col].mode()[0])

# Numerical basement-related: fill with 0
numerical_fill_zero = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']
for col in numerical_fill_zero:
    test_df[col] = test_df[col].fillna(0)

# Verify no remaining missing values
missing_test_final = test_df.isnull().sum()
missing_test_final[missing_test_final > 0]


Series([], dtype: int64)

#### creating the same features as we have done for the train dataset.

In [14]:
# Feature: House Age
test_df['HouseAge'] = test_df['YrSold'] - test_df['YearBuilt']

# Feature: Remodeled flag
test_df['Remodeled'] = (test_df['YearBuilt'] != test_df['YearRemodAdd']).astype(int)

# Feature: Garage Age
test_df['GarageAge'] = test_df['YrSold'] - test_df['GarageYrBlt']

# Feature: Total Bathrooms
test_df['TotalBath'] = (
    test_df['FullBath'] +
    0.5 * test_df['HalfBath'] +
    test_df['BsmtFullBath'] +
    0.5 * test_df['BsmtHalfBath']
)

# Feature: Total Porch SF
test_df['TotalPorchSF'] = (
    test_df['OpenPorchSF'] +
    test_df['EnclosedPorch'] +
    test_df['3SsnPorch'] +
    test_df['ScreenPorch']
)

# Binary flags
test_df['HasPool'] = (test_df['PoolArea'] > 0).astype(int)
test_df['HasFireplace'] = (test_df['Fireplaces'] > 0).astype(int)
test_df['HasGarage'] = (test_df['GarageType'] != 'None').astype(int)

# Show sample
test_df[['HouseAge', 'Remodeled', 'GarageAge', 'TotalBath', 'TotalPorchSF', 'HasPool', 'HasFireplace', 'HasGarage']].head()


,HouseAge,Remodeled,GarageAge,TotalBath,TotalPorchSF,HasPool,HasFireplace,HasGarage
0,49,0,49.0,1.0,120,0,0,1
1,52,0,52.0,1.5,36,0,0,1
2,13,1,13.0,2.5,34,0,1,1
3,12,0,12.0,2.5,36,0,1,1
4,18,0,18.0,2.0,226,0,0,1


In [15]:
# Apply ordinal mapping
for col in ordinal_cols:
    test_df[col] = test_df[col].map(quality_mapping)

# Show a sample to confirm
test_df[ordinal_cols].head()


,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond,PoolQC
0,3,3,3,3,3,3.0,0,3,3,0
1,3,3,3,3,3,4.0,0,3,3,0
2,3,3,4,3,4,3.0,3,3,3,0
3,3,3,3,3,5,4.0,4,3,3,0
4,4,3,4,3,5,4.0,0,3,3,0


In [16]:
# One-hot encode nominal columns in test data (same columns as train)
test_df_encoded = pd.get_dummies(test_df, columns=nominal_cols, drop_first=True)

# Align test columns to match train columns
# Any missing columns will be added with 0
test_df_encoded = test_df_encoded.reindex(columns=train_df_encoded.columns, fill_value=0)

# Note: SalePrice exists in train but not test, so we'll remove it here if present
if 'SalePrice' in test_df_encoded.columns:
    test_df_encoded = test_df_encoded.drop('SalePrice', axis=1)

# Show shape and sample columns
(test_df_encoded.shape, test_df_encoded.columns[:10])


((1459, 208),
 Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'Alley', 'LandSlope',
        'Condition2', 'OverallQual', 'OverallCond', 'YearBuilt'],
       dtype='object'))

### Scaling Numerical Features
Finally, scale numeric columns using the same scaler fitted on the training set.

In [17]:
# Scale numeric columns in test set
test_df_encoded[numeric_cols_to_scale] = scaler.transform(test_df_encoded[numeric_cols_to_scale])

# Show a sample
test_df_encoded[numeric_cols_to_scale].head()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,YrSold,HouseAge,GarageAge,TotalBath,TotalPorchSF
0,-0.872563,0.460320,0.110763,-0.795151,0.381743,-0.340077,-1.156380,-0.570750,-0.689604,-0.238112,...,1.882709,-0.068692,-0.066236,-0.087688,-0.119110,1.64521,0.411777,0.671560,-1.541932,0.313017
1,-0.872563,0.505733,0.375850,-0.071836,0.381743,-0.439440,-1.301740,0.027027,-0.689604,-0.238112,...,-0.270208,-0.068692,-0.066236,25.116309,-0.119110,1.64521,0.510984,0.785436,-0.905095,-0.485809
2,0.073375,0.187842,0.332053,-0.795151,-0.517200,0.852269,0.636400,-0.570750,-0.689604,-0.238112,...,-0.270208,-0.068692,-0.066236,-0.087688,-1.229111,1.64521,-0.778707,-0.694959,0.368581,-0.504829
3,0.073375,0.369494,-0.054002,-0.071836,0.381743,0.885390,0.636400,-0.460051,-0.689604,-0.238112,...,-0.270208,-0.068692,-0.066236,-0.087688,-0.119110,1.64521,-0.811776,-0.732918,0.368581,-0.485809
4,1.492282,-1.219961,-0.552407,1.374795,-0.517200,0.686666,0.345679,-0.570750,1.052302,-0.238112,...,2.313293,-0.068692,-0.066236,-0.087688,-1.969111,1.64521,-0.613362,-0.505165,-0.268257,1.321059
